# Detect sources from sources Light Curves

- author Sylvie Dagoret-Campagne
- creation date 2024-05-30
- last update 2024-05-31
- affiliation : IJCLab
- Kernel **w_2024_16**


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

In [ ]:
import gc

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
# LSST Science Pipeline packages (see pipelines.lsst.io)
import lsst.daf.base as dafBase
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom



# Pipeline tasks
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.detection import SourceDetectionTask
from lsst.meas.deblender import SourceDeblendTask
from lsst.meas.base import SingleFrameMeasurementTask

In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')
plt.rcParams['figure.figsize'] = (8.0, 8.0)
plt.style.use('tableau-colorblind10')

In [ ]:
from astropy.modeling import models, fitting
from astropy import modeling
# define a model for a line
g_init = models.Gaussian1D(amplitude=1, mean=0, stddev=1)
# initialize a linear fitter
fit_g = fitting.LevMarLSQFitter()

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

Import statements that we will need later

Let's make a new plot and metric tool, we'll base it on the example in the getting started guide.

In [ ]:
# INSERT YOUR collection and tract
# for rehearsal use collection 2 which have CCDvisit
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
#collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240311/w_2024_10/PREOPS-4985'
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240414/w_2024_15/PREOPS-5069' # COMPLETED

collectionn = collection
#collections = [collection1,collection2,collection3]
collections = [collection]
collectionStr = collectionn.replace("/", "_")
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"
instrument = 'LATISS'
skymapName = "latiss_v1"
where_clause = "instrument = \'" + instrument+ "\'"
tract = 3864 # mostly for light-curves
patch_sel = 236
band = 'g'
#tract = 5615
# tract = 5634 # interesting to view calib parameters
suptitle = collectionStr + f" inst = {instrument} tract = {tract}"

In [ ]:
#dataId = {"skymap": "latiss_v1", "tract": 5615, "instrument": "LATISS"}
dataId = {"skymap": skymapName, "tract": tract, "instrument": instrument}
repo = '/sdf/group/rubin/repo/oga/'
butler = dafButler.Butler(repo)
#t = Butler.get(table_sel, dataId=dataId, collections=collections)
registry = butler.registry

In [ ]:
skymap_auxtel ='latiss_v1'
skymap = butler.get('skyMap', skymap=skymap_auxtel, collections=collections)

In [ ]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

## Get Pixel Scale

In [ ]:
import lsst.geom as geom
import lsst.sphgeom

skymap = butler.get('skyMap', skymap=skymapName, collections=collections )
tractInfo = skymap.generateTract(tract)
for patch in tractInfo:    
    patchID = patch.getSequentialIndex()
        
    ibb=patch.getInnerBBox()
    tWCS=tractInfo.getWcs()
       
    # loop on the 4 corners
    for icorn,corner in enumerate(ibb.getCorners()):
        p = geom.Point2D(corner.getX(), corner.getY())
        coord = tWCS.pixelToSky(p)

In [ ]:
tWCS

In [ ]:
#arcsec/pixel
pixel_scale = tWCS.getPixelScale().asArcseconds()

## Selected visits

In [ ]:
inputfilename = "sources_objectTable-t3864-bg-o547-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"
#inputfilename = "sources_objectTable-t3864-bg-o912-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"
df_myselectedvisits = pd.read_csv(inputfilename ,index_col=0)
tract = 3864
band = "g"
objectname = 547
title = f"Auxtel Light Curves : tract = {tract}, band = {band}, object = {objectname} "
suptitle = inputfilename 

In [ ]:
df_myselectedvisits.sort_values("visit",inplace=True)
#df_myselectedvisits.sort_index(inplace=True)

In [ ]:
df_myselectedvisits

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(14,4))
df_myselectedvisits.plot.scatter(x="expMidptMjd",y="psfMag",ax=ax,s=20,c="zeroPoint",cmap="jet",grid=True,rot=45)
ax.set_title(title)
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
df_myselectedvisits["visit"]

Note here:
- zeroPoint is in ABMag
- skyBg is in ADU (but don't know if in pixel or in arcsec2)
- skyNoise is in ADU (but don't know if in pixel or in arcsec2)

In [ ]:
index = 0
visitId = df_myselectedvisits.iloc[index]['visit']

In [ ]:
df_myselectedvisits.iloc[index]

In [ ]:
x_target = df_myselectedvisits.iloc[index]['x']
y_target = df_myselectedvisits.iloc[index]['y']

In [ ]:
datasetType = 'calexp'
dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
datasetRefs = registry.queryDatasets(datasetType, dataId=dataId, collections  = collections)

for i, ref in enumerate(datasetRefs):
    print(ref.dataId)
    print("band:", ref.dataId['band'])
    band = ref.dataId['band']

## One Calexp 

In [ ]:
calexp = butler.get('calexp', **dataId,collections=collections)
mask = calexp.mask.array

In [ ]:
the_mask = np.where(mask==0,1,0)
the_mask.shape

In [ ]:
im = plt.imshow(the_mask,origin="lower",cmap="grey")
plt.colorbar(im)

In [ ]:
img = calexp.image.array * (the_mask)

In [ ]:
im=plt.imshow(img,origin="lower",vmin=-10,vmax=10,cmap="grey")
plt.colorbar(im)

In [ ]:
#%matplotlib widget

calexp = butler.get('calexp', **dataId,collections=collections)

calexp_info = calexp.getInfo()
photocalib = calexp_info.getPhotoCalib()
expo_photocalibconstant_mean = photocalib.getCalibrationMean()
expo_photocalibconstant_error = photocalib.getCalibrationErr()
calexp_md = calexp_info.getMetadata()
magzero,magzero_rms,magzero_nobj = calexp_md["MAGZERO"],calexp_md["MAGZERO_RMS"],calexp_md["MAGZERO_NOBJ"] 
bgmean,bgvar = calexp_md["BGMEAN"],calexp_md["BGVAR"]

mask = calexp.mask.array
the_mask = np.where(mask==0,1,0)
#img = calexp.maskedImage.getImage()
img = calexp.image.array 
#img = calexp[the_mask] 
wcs = calexp.getWcs()
the_fits_WCS = WCS(wcs.getFitsMetadata())
#bbox = img.getBBox()
#corners_ra, corners_dec = get_corners_radec(wcs, bbox)
#extent = [np.min(corners_ra),np.max(corners_ra),np.min(corners_dec),np.max(corners_dec)]


# Display the image with a suitable scaling
data_flat = img.flatten()
mean = np.mean(data_flat)
med = np.median(data_flat)
sigMad = 1.4826 * np.median(np.fabs(data_flat - mean))
#vmin = med - 3 * sigMad
#vmax = med + 3 * sigMad
vmin = med - 50.
vmax = med + 50.
the_min= data_flat.min()
the_max= data_flat.max()
print(mean,med ,sigMad ,vmin ,vmax,the_min,the_max )

fig,axs = plt.subplots(1,2,figsize=(14,6))

## Plot the image
ax = axs[0]
from astropy.visualization import ZScaleInterval
z = ZScaleInterval()
z1,z2 = z.get_limits(img)
from matplotlib.colors import SymLogNorm
im=ax.imshow(img, origin="lower", cmap="grey",norm=SymLogNorm(linthresh=0.1))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
the_title = f"Calexp : visit = {visitId}, band = {band}"
ax.set_title(the_title)
ax.grid()
ax.set_aspect('equal')
plt.colorbar(im, cax=cax,ax=ax)



ax = axs[1]

#val = data_flat/med)
#data_flat_mag = -2.5*np.log10((data_flat/med))*1000
#mu_mag = np.mean(data_flat_mag)
#med_mag = np.median(data_flat_mag)
#sigMad_mag = 1.4826 * np.median(np.fabs(data_flat_mag - med_mag))
#vmin_mag = med_mag - 3 * sigMad_mag
#vmax_mag = med_mag + 3 * sigMad_mag
#textstr = '\n'.join((
#    r'$\mu=%.2f$ mmag' % (mu_mag, ),
#    r'$\mathrm{med}=%.2f$ mmag' % (med_mag, ),
#    r'$\sigma=%.2f$ mmag' % (sigMad_mag , )))

data_flat_nozero = data_flat[~(data_flat==0)]
histdata  = ax.hist(data_flat_nozero,bins=100,range=(vmin,vmax),histtype="step",color="b",lw=2)
histarray,histedges = histdata[0],histdata[1]
ax.set_ylim(0.,histarray.max()*1.1)

# fit
g = fit_g(g_init,histedges[1:],histarray)
print("fig_g parameters: ",g)
m = modeling.models.Gaussian1D(amplitude=g.amplitude.value, mean=g.mean.value, stddev=g.stddev.value)
ax.plot(histedges,m(histedges),"r-")
textstr = '\n'.join((
    r'$\mu=%.2f$ ADU/pixel' % (g.mean.value, ),
    r'$\sigma=%.2f$ ADU/pixel' % (g.stddev.value , ),
    r'$\sigma=%.2f$ mag/s/arcsec2' % (convert_fluxtomag(g.stddev.value*expo_photocalibconstant_mean/30./pixel_scale**2) ,)))
props = dict(boxstyle='round', facecolor='wheat' ,alpha=0.5)
ax.text(0.05, 0.95, textstr, color="r",transform=ax.transAxes, fontsize=10,verticalalignment='top', bbox=props)
ax.set_title("Calexp Sky-bkg residuals")
ax.set_xlabel("Flux (ADU)")

#ax.grid()

plt.suptitle(collections)
plt.tight_layout()
plt.show()
#remove_figure(fig)

## Check the wcs from rubin

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(projection=the_fits_WCS)
calexp_extent = (calexp.getBBox().beginX, calexp.getBBox().endX,
                 calexp.getBBox().beginY, calexp.getBBox().endY)
im = ax.imshow(calexp.image.array, cmap='gray', vmin=vmin, vmax=vmax,extent=calexp_extent, origin='lower')
ax.grid(color='white', ls='solid')
ax.set_xlabel('Right Ascension')
ax.set_ylabel('Declination')
overlay = ax.get_coords_overlay('icrs')
overlay.grid(color='white', ls='dotted')
the_title = f"Calexp : visit = {visitId}, band = {band}"
ax.set_title(the_title)
plt.show()
#remove_figure(fig)

## Copy DP0 tuto

<br>

As described in other tutorials, the `calexp` object possesses more than just the raw pixel data of the image. It also contains a `mask`, which stores information about various pixels in a bit mask.

Here are some optional commands to explore the calexp. Uncomment one of the code lines to learn more.

In [ ]:
# If you want to investigate the contents of the masked image:
# calexp.maskedImage

# If you just want one of the three components:
# calexp.maskedImage.image
# calexp.maskedImage.mask
# calexp.maskedImage.variance

# These also work:
# calexp.image
# calexp.mask
# calexp.variance

# The calexp also contains the PSF, the WCS, and the photometric calibration
# calexp.getPsf()
# calexp.getWcs()
# calexp.getPhotoCalib()

Since we are interested in performing our own source detection and measurement, we choose to clear the existing `DETECTED` mask plane.

In [ ]:
# Unset the `DETECTED` bits of the mask plane
calexp.mask.removeAndClearMaskPlane('DETECTED')

In [ ]:
# Plot the calexp we just retrieved
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale',cmap="Grays")
afw_display.mtv(calexp.image)

<br>

### 2.2. Add the Subtracted Sky Background Back into the Image

Here we retrieve the subtracted background for the same dataId and add it back into the image. This section is optional.

First, we obtain the `calexpBackground` object for this `dataId`.  We will again use the `butler`.

In [ ]:
dataId

In [ ]:
bkgd = butler.get('calexpBackground', **dataId,collections=collections)

In [ ]:
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('linear', 'zscale')
afw_display.mtv(bkgd.getImage())
plt.title("Local Polynomial Background")

Next, we add the background into the `calexp`, and re-display the `calexp`. Note the scale in the sidebar now goes up to thousands of counts instead of hundreds of counts.

In [ ]:
# Note: executing this cell multiple times will add the background
#  multiple times
calexp.maskedImage += bkgd.getImage()

In [ ]:
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(calexp.image)

## 3. Source Detection, Deblending, and Measurement

We now want to run the LSST Science Pipelines' source detection, deblending, and measurement tasks. While we run all three tasks, this notebook is mostly focused on the detection of sources.

Recall that these tasks were imported up at the top of this notebook, from `lsst.pipe` and `lsst.meas`. More information can be found at [pipelines.lsst.io](https://pipelines.lsst.io/) (the search bar at the top left of that page is a very handy way to find documentation for a specific task).

We start by creating a minimal schema for the source table. The schema describes the output properties that will be measured for each source. This schema will be passed to all of the tasks, as we call each in turn, and each task will add columns to this schema as it measures sources in the image.

In [ ]:
# Create a basic schema to use with these tasks
schema = afwTable.SourceTable.makeMinimalSchema()
print(schema)

# Create a container which will be used to record metadata
#  about algorithm execution
algMetadata = dafBase.PropertyList()
print('algMetadata: ')
algMetadata

### 3.1. Configuring Tasks

Each task possesses an associated configuration class. The properties of these configuration classes can be determined from the classes themselves.

In [ ]:
# Uncomment the following line to view help
#  for the CharacterizeImageTask configuration
# Replace 'CharacterizeImageTask' with a different
#  task name to view additional help information
#help(CharacterizeImageTask.ConfigClass())

As a starting point, like the `schema` and `algMetadata` above, here we set some basic config parameters and instantiate the tasks to get you started. In this case, we configure several different tasks:

* CharacterizeImageTask: Characterizes the image properties (e.g., PSF, etc.)
* SourceDetectionTask: Detects sources
* SourceDeblendTask: Deblend sources into constituent "children"
* SingleFrameMeasurementTask: Measures source properties

In [ ]:
# Characterize the image properties
config = CharacterizeImageTask.ConfigClass()
config.psfIterations = 3
charImageTask = CharacterizeImageTask(config=config)

# Detect sources
config = SourceDetectionTask.ConfigClass()
# detection threshold in units of thresholdType
config.thresholdValue = 10
# units for thresholdValue
config.thresholdType = "stdev"
sourceDetectionTask = SourceDetectionTask(schema=schema, config=config)

# Deblend sources
sourceDeblendTask = SourceDeblendTask(schema=schema)

# Measure source properties
config = SingleFrameMeasurementTask.ConfigClass()
sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema,
                                                   config=config,
                                                   algMetadata=algMetadata)

Note that if you want to change the value of a config parameter (e.g., `psfIterations`), do not change it in the already constructed task. Instead, change your config object and then construct a new characterize image task. Like so:
> `config.psfIterations = 3` <br>
> `charImageTask = CharacterizeImageTask(config=config)`

Like the configs, we can use `help` to explore each task and the methods used to run it.

In [ ]:
# help(charImageTask)

# Uncomment the following line, position your cursor after the period,
#  and press tab to see a list of all methods. Then recomment the line
#  because "Task." is not executable and will cause an error.
# charImageTask.

# Use the help function on any of the methods to learn more:
# help(charImageTask.writeSchemas)

# E.g., find out what options there are for config.thresholdType
# help(SourceDetectionTask.ConfigClass)

With each of the tasks configured, we can now move on to running the source detection, deblending, and measurement. First we create `SourceTable` for holding the output of our source analysis. The columns and characteristics of this table are defined by the `schema` that we created in our configuration step.

In [ ]:
tab = afwTable.SourceTable.make(schema)

### 3.2. Image Characterization

Next we characterize our image. This calculates various global properties of the image, such as the full-width half-max of its point spread function (PSF FWHM).

In [ ]:
# Image characterization (this cell may take a few seconds)
result = charImageTask.run(calexp)

# Define the pixel coordinates of a point of interest
# (in this case, basically a random point within the image)
#x_target, y_target = 1700, 2100
width, height = 400, 400
xmin, ymin = x_target-width//2, y_target-height//2
point = geom.Point2D(x_target, y_target)

# Get the PSF at our point of interest
psf = calexp.getPsf()
sigma = psf.computeShape(point).getDeterminantRadius()
pixelScale = calexp.getWcs().getPixelScale().asArcseconds()

# The factor of 2.355 converts from std to fwhm
print('psf fwhm = {:.2f} arcsec'.format(sigma*pixelScale*2.355))

With the image characterized, we are now interested in running the source detection, deblending, and measurement tasks. Each of these tasks is called with the `run` method. The parameters of this method can be investigated using `help`.

In [ ]:
# We are specifically interested in the `SourceMeasurementTask`
# help(sourceMeasurementTask.run)

In [ ]:
# Source detection (this cell may take a few seconds)
result = sourceDetectionTask.run(tab, calexp)
type(result)

The source detection task returns an [`lsst.pipe.base.struct.Struct`](http://doxygen.lsst.codes/stack/doxygen/x_masterDoxyDoc/classlsst_1_1pipe_1_1base_1_1struct_1_1_struct.html). A `Struct` is just a generalized container for storing multiple output components and accessing them as attributes. The content of this `Struct` can be investigated with the `getDict` method.

In [ ]:
for k, v in result.getDict().items():
    print(k, type(v))

In [ ]:
result.numPosPeaks

In [ ]:
sources = result.sources

In [ ]:
# Uncomment to see that most values are NaN
# sources

Note that if we desire we can save these processed sources to disk as FITS tables:

In [ ]:
# sources.writeFits("detectedsources-outputTable.fits")
# calexp.writeFits("calexp-detectedsources-example1-out.fits")

### 3.3. Deblend and Measure Sources

Next we run the `SourceDeblendTask` and `SingleFrameMeasurementTask`. A deeper investigation of these tasks is beyond the scope of this notebook.

In [ ]:
# Source deblending
sourceDeblendTask.run(calexp, sources)

# Source measurement
sourceMeasurementTask.run(measCat=sources, exposure=calexp)

In [ ]:
# The copy makes sure that the sources are sequential in memory
sources = sources.copy(True)

# Investigate the output source catalog
sources.asAstropy()

In [ ]:
#sources.columns.get("ApInstFlux")


In [ ]:
sources.asAstropy().to_pandas()

In [ ]:
list(zip(sources.columns.getX(),sources.columns.getY(),sources.getApInstFlux(),sources.getApInstFluxErr()))

### 3.4. Overplot Sources on Image

We can now overplot our detected sources on the calexp or a cutout image using `afwDisplay`.

Let's do this for a cutout image instead of the full calexp.  To generate a cutout image of the calexp, create a bounding box and pass it to the `Factory` method of our calexp (a `lsst.afw.image.Exposure` object). Below we explain the specific arguments that we are passing to `Factory`:
```
calexp : the ExposureF we are starting from
bbox   : the bounding box of the cutout
origin : the image pixel origin is local to the cutout array
deep   : copy the data rather than passing by reference
```

<a id='display-error'></a>

In [ ]:
# Define a small region for a cutout
bbox = geom.Box2I()
bbox.include(geom.Point2I(xmin, ymin))
bbox.include(geom.Point2I(xmin + width, ymin + height))

# An alternative way to defined the same cutout region
# bbox = geom.Box2I(geom.Point2I(xmin, ymin), geom.Extent2I(width, height))

# Generate the cutout image
cutout = calexp.Factory(calexp, bbox, origin=afwImage.LOCAL, deep=False)
extent = (xmin,ymin,xmin+width,ymin+height)

In [ ]:
xmin

In [ ]:
# Display the cutout and sources with afw display
image = cutout.image
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(image)
#plt.gca().axis('off')

# We use display buffering to avoid re-drawing the image
#  after each source is plotted
with afw_display.Buffering():
    for s in sources:
        afw_display.dot('+', s.getX(), s.getY(), ctype=afwDisplay.RED)
        afw_display.dot('o', s.getX(), s.getY(), size=20, ctype='orange')

## 4. Footprints

Object footprints are an integral component of the high-level CCD processing tasks (e.g., detection, measurement, and deblending). To quote [Bosch et al. (2017)](https://arxiv.org/pdf/1705.06766.pdf), 

> Footprints record the exact above-threshold detection region on a CCD. These are similar to  SExtractor’s “segmentation map", in that they identify which pixels belong to which detected objects

This quote draws an analogy between footprints and segmentation maps, since they both identify pixels with values above some threshold. This is a useful similarity, since it gives us a place to start understanding the properties of footprints.

The result of the `SourceDetectionTask` stores the footprints associated to detected objects.

In [ ]:
# Grab the above-threshold footprints that were detected,
#  and assign them to the variable `fps`
fpset = result.positive
fps = fpset.getFootprints()

In [ ]:
# We can get a rough view of the first source's footprint from its span
fps[0].getSpans()

You can almost see the footprint by looking at the 1's and 0's here. Keep in mind that the first row of this array will be the *bottom* row of the image. Later, when we display the footprint, its general pattern will appear "upside down" compared to this pattern of 1s. 

### 4.1. Heavy Footprints

At the moment, our footprints indicate which pixels they consist of, but not the values of those pixels from the image. To extract the actual values of the pixels that correspond to the ones in the footprint span, we need to convert our footprint into a `HeavyFootprint`. HeavyFootprints have all of the qualities of Footprints, but additionally 'know' about pixel level data from the image, variance, and mask planes.

In [ ]:
# First we demonstrate that the footprint is NOT heavy
fps[0].isHeavy()

In [ ]:
# Next, we make all the footprints heavy at the same time
#  by operating on the footprint set
fpset.makeHeavy(calexp.getMaskedImage())

# This means we have to redefine fps:
hfps = fpset.getFootprints()

In [ ]:
# All of the arrays here will be flattened 1D arrays
#  of pixels from the footprint.
# Uncomment this line to print the array and see that
#  now, it contains pixel values.

# hfps[0].getImageArray()

Now we can use the span set to reassemble the image array into the footprint. Above we saw that the image array is a 1D numpy array, but the footprint itself is 2 dimensional. Fortunately, the span set has an `unflatten` method that rearranges the image array into the proper 2 dimensional shape. If you want to change the colormap, see [matplotlib colormap options](https://matplotlib.org/stable/tutorials/colors/colormaps.html).

In [ ]:
plt.figure()
plt.imshow(fps[0].getSpans().unflatten(hfps[0].getImageArray()),
           cmap='bone', origin='lower')

In [ ]:
hfps[0].getMaskArray()

In [ ]:
calexp.getMask().getMaskPlaneDict()

The values are the exponent of the bitmask. So pixels only marked detected will be 2^5 = 32. Pixels that are both on the edge of the original image and detected will be 2^5 + 2^4 = 48. We will visualize the mask plane values in a similar manner as before, except that we will be displaying the values of the mask array.

In [ ]:
plt.figure()
im = plt.imshow(fps[0].getSpans().unflatten(hfps[0].getMaskArray()),
                origin='lower')

# Create a new axis, "cax" on the right side of the image display.
# The width of cax will be 5% of the axis "ax".
# The padding between cax and ax will be 1% of the axis.
ax = plt.gca()
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad="1%")
plt.colorbar(im, cax=cax, ticks=[0, 32, 32+16])